In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
# from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
#import config
import quickstart

In [ ]:
df= pd.read_csv('fire_dispatch.csv').drop("Unnamed: 0", axis = 1).rename({"BoroCT2010":"boroct2010"}, axis = 1)
df["boroct2010"]= df["boroct2010"].astype(str)
df.head()

In [ ]:
READ CENSUS DATA

In [ ]:
labels_to_drop = ["Unnamed: 0", "Unnamed: 0.3", "Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0.1.1", 'pct18under',
       'pct18_34', 'pct35_64', 'pct65plus', 'pcthispanic', 'pctnhwhite',
       'pctnhblack', 'pctnhaian', 'pctnhasian', 'pctnhhawainpi', 'pctnhother',
       'pctnhtwoplus', 'pctownerocch', 'pctrentocch', 'crowding',
       'crowdingrenter']
census_tract = census_tract.drop(labels_to_drop, axis = 1)
census_tract = census_tract[census_tract['year'] == 2019]
census_tract = census_tract.fillna(0)

# make region key the same as crosswalk file
def get_region(x):
    y = int(str(x)[:5])
    z = str(x)[5:]
    match y:
        case (36005):
            return int("2" + z)
        case (36047):
            return int("3" + z)
        case (36061):
            return int("1" + z)
        case (36081):
            return int("4" + z)
        case (36085):
            return int("5" + z)
census_tract["geofips"] = census_tract["geofips"].apply(lambda x : get_region(x))

census_tract

In [ ]:
crosswalk = pd.read_csv('ct_box_xwalk1.csv')
crosswalk = crosswalk.sort_values(by="borobox")

In [ ]:
columnNames = ['borobox', 'count', 'totpop', 'popdens', 'areasqmile', 'males', 'females',
       'pop18under', 'pop18_34', 'pop35_64', 'pop65plus', 'pophispanic',
       'popnhwhite', 'popnhblack', 'popnhaian', 'popnhasian', 'popnhhawainpi',
       'popnhother', 'popnhtwoplus', 'housing', 'housingoccupied',
       'renterhousing', 'oc_halfless', 'oc_half_1', 'oc_1_onehalf',
       'oc_onehalf_2', 'oc_2plus', 'roc_halfless', 'roc_half_1',
       'roc_1_onehalf', 'roc_onehalf_2', 'roc_2plus', 'medianyearbuilt',
       'mediangrossrent', 'medianhhincome', 'gini', 'year']
weighted_borobox = pd.DataFrame(columns = columnNames)
numRuns = 0

for index, row in crosswalk.iterrows():
    box = row["borobox"]
    weight = row["wgt"]
    try:
        tract = int(row["boroct2010"])
    except:
        continue

    try: 
        
        row_census = census_tract.loc[census_tract['geofips'] == tract].iloc[-1]

        # ------------ in order to test with smaller datset, uncomment the below three lines ---------
        # numRuns += 1
        # if numRuns > 10:
        #     break
        # ------------ end test section ---------------

# ---------add new row with borobox specified if row does not exist, otherwise edit existing row
        try:            # find existing row
            row_weighted = weighted_borobox.loc[weighted_borobox["borobox"] == box] # make copy of existing data to act on it
            row_weighted.at[0,'count'] = row_weighted.at[0,'count'] + 1
            for i, column in enumerate(columnNames):
                if i<2:
                    continue
                elif (i<32):
                    row_weighted.at[0,column] = (row_weighted.at[0,column] + (row_census[column] * weight))
                else:
                    row_weighted.at[0,column] = row_weighted.at[0,column] + row_census[column]
            weighted_borobox.loc[weighted_borobox["borobox"] == box] = row_weighted # paste in finalized new data

        except:         # create new row
            # create a column that initiates at one, if column is accessed again, increment. add up some columns that require averages. at end, divide to find target numbers.
            row_weighted = pd.DataFrame(columns = columnNames)
            row_weighted.at[0,'count'] = 1
            row_weighted.at[0,'borobox'] = box
            for i, column in enumerate(columnNames):
                if i<2:
                    continue
                elif (i<32):
                    row_weighted.at[0,column] = (row_census[column] * weight)
                else:
                    row_weighted.at[0,column] = row_census[column]
            weighted_borobox = pd.concat([weighted_borobox,row_weighted])
    except:
        print("error number: ")
        continue

In [ ]:
weighted_borobox.sort_values(by='medianhhincome')
weighted_borobox = weighted_borobox.reset_index()
weighted_borobox

In [ ]:
for index, row in weighted_borobox.iterrows():
    instances = int(row["count"])
    # print(row.at['medianyearbuilt'])
    row.at['medianyearbuilt'] = (row.at['medianyearbuilt']/instances)
    row.at['mediangrossrent'] = (row.at['mediangrossrent']/instances)
    row.at['medianhhincome'] = (row.at['medianhhincome']/instances)
    row.at['gini'] = (row.at['gini']/instances)
    row.at['year'] = (row.at['year']/instances)

In [ ]:
for index_label, row_series in weighted_borobox.iterrows():
       # print(index_label)
       # if indexNum > 10:
       #        break
       totalPop = row_series.at['totpop']
       weighted_borobox.loc[index_label,'pct18under'] = ((row_series.at['pop18under']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct18_34'] = ((row_series.at['pop18_34']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct35_64'] = ((row_series.at['pop35_64']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct65plus'] = ((row_series.at['pop65plus']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pcthispanic'] = ((row_series.at['pophispanic']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhwhite'] = ((row_series.at['popnhwhite']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhblack'] = ((row_series.at['popnhblack']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhaian'] = ((row_series.at['popnhaian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhasian'] = ((row_series.at['popnhasian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhhawainpi'] = ((row_series.at['popnhhawainpi']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhother'] = ((row_series.at['popnhwhite']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhtwoplus'] = ((row_series.at['popnhtwoplus']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhaian'] = ((row_series.at['popnhaian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctownerocch'] = (row_series['housingoccupied'] / row_series['housing'])*100
       weighted_borobox.loc[index_label,'pctrentocch'] = (row_series['renterhousing'] / row_series['housing'])*100

In [ ]:
weighted_borobox.to_csv('weighted_borobox.csv')